# Multi-Method Transit Detection

Compare BLS, GLS, and PDM methods with consensus weighting.

**Topics:** Multiple detection algorithms, consensus period, harmonics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.core import (
    generate_transit_signal_mandel_agol,
    add_noise,
    find_transits_multiple_methods,
    find_transits_bls_advanced,
    find_period_gls,
    find_period_pdm
)

## 1. Generate Test Data

In [ ]:
TRUE_PERIOD = 3.5
TRUE_DEPTH = 0.008

time = np.linspace(0, 50, 3000)
flux = generate_transit_signal_mandel_agol(time, period=TRUE_PERIOD, depth=TRUE_DEPTH)
flux = add_noise(flux, noise_level=0.0015, seed=123)

plt.figure(figsize=(12, 3))
plt.plot(time, flux, 'k.', ms=1)
plt.xlabel('Time (days)')
plt.ylabel('Flux')
plt.title(f'Test Light Curve (P={TRUE_PERIOD} d, depth={TRUE_DEPTH*100:.1f}%)')
plt.show()

## 2. Run Individual Methods

In [ ]:
# BLS
bls_result = find_transits_bls_advanced(time, flux, min_period=1, max_period=15)
print(f"BLS:  P = {bls_result['period']:.4f} d, SNR = {bls_result['snr']:.1f}")

# GLS
gls_result = find_period_gls(time, flux)
print(f"GLS:  P = {gls_result['period']:.4f} d")

# PDM
pdm_result = find_period_pdm(time, flux, min_period=1, max_period=15)
print(f"PDM:  P = {pdm_result['period']:.4f} d, theta = {pdm_result['theta']:.3f}")

## 3. Multi-Method with Consensus

In [ ]:
# Run all methods with consensus
results = find_transits_multiple_methods(
    time, flux,
    min_period=1.0,
    max_period=15.0,
    methods=['bls', 'gls', 'pdm']
)

print('\n' + '='*50)
print('Multi-Method Results')
print('='*50)
print(f"BLS Period:       {results['bls']['period']:.4f} d")
print(f"GLS Period:       {results['gls']['period']:.4f} d")
print(f"PDM Period:       {results['pdm']['period']:.4f} d")
print(f"\nConsensus Period: {results['consensus']['period']:.4f} d")
print(f"Period Std:       {results['consensus']['period_std']:.4f} d")
print(f"True Period:      {TRUE_PERIOD:.4f} d")

## 4. Compare Periodograms

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# BLS
axes[0].plot(results['bls']['all_periods'], results['bls']['all_powers'], 'b-', lw=0.5)
axes[0].axvline(TRUE_PERIOD, color='g', ls='--', alpha=0.7)
axes[0].axvline(results['bls']['period'], color='r', ls=':')
axes[0].set_ylabel('BLS Power')
axes[0].set_title('BLS Periodogram')

# GLS
axes[1].plot(1/results['gls']['frequencies'], results['gls']['powers'], 'orange', lw=0.5)
axes[1].axvline(TRUE_PERIOD, color='g', ls='--', alpha=0.7)
axes[1].axvline(results['gls']['period'], color='r', ls=':')
axes[1].set_ylabel('GLS Power')
axes[1].set_title('GLS Periodogram')
axes[1].set_xlim(1, 15)

# PDM
axes[2].plot(results['pdm']['periods'], results['pdm']['thetas'], 'purple', lw=0.5)
axes[2].axvline(TRUE_PERIOD, color='g', ls='--', alpha=0.7, label='True')
axes[2].axvline(results['pdm']['period'], color='r', ls=':', label='Detected')
axes[2].set_ylabel('PDM Theta')
axes[2].set_xlabel('Period (days)')
axes[2].set_title('PDM (lower = better)')
axes[2].legend()

plt.tight_layout()
plt.show()

## 5. Method Comparison Table

In [ ]:
# Summary table
print('\nMethod Comparison:')
print('-' * 45)
print(f'{"Method":<12} {"Period (d)":<12} {"Error (%)":<12}')
print('-' * 45)

for method in ['bls', 'gls', 'pdm']:
    p = results[method]['period']
    err = abs(p - TRUE_PERIOD) / TRUE_PERIOD * 100
    print(f'{method.upper():<12} {p:<12.4f} {err:<12.2f}')

cons_p = results['consensus']['period']
cons_err = abs(cons_p - TRUE_PERIOD) / TRUE_PERIOD * 100
print('-' * 45)
print(f'{"CONSENSUS":<12} {cons_p:<12.4f} {cons_err:<12.2f}')
print('-' * 45)

## Summary

Multi-method detection provides:
- **Robustness**: Cross-validation between methods
- **Confidence**: Consensus period with uncertainty
- **Harmonic rejection**: Identify false periods

**When to use which:**
- BLS: Best for box-shaped transits
- GLS: Good for sinusoidal/ellipsoidal variations
- PDM: Robust for irregular/non-sinusoidal signals